# Text-to-SQL with Gemini 2.0

see similar code example: [code execution with Gemini 2.0](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/code-execution/intro_code_execution.ipynb)


In [21]:
# %pip install --upgrade --quiet google-genai

### Imports

see `google-genai` SDK [here](https://github.com/googleapis/python-genai/tree/main/google/genai)

In [6]:
import os
import sys

from IPython.display import Markdown, display

from google import genai
from google.genai.types import GenerateContentConfig

### environment config

In [2]:
PROJECT_ID = "hybrid-vertex" 
LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    raise ValueError("Please set your PROJECT_ID")
    
print(f"PROJECT_ID: {PROJECT_ID}")
print(f"LOCATION: {LOCATION}")

PROJECT_ID: hybrid-vertex
LOCATION: us-central1


In [3]:
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

In [4]:
MODEL_ID = "gemini-2.0-flash-001"

# Generate SQL for BigQuery

> generate SQL queries to test against a BigQuery Public dataset: 

In the SQL prompt template, you can define multiple criteria like:

* problem statement
* table metadata
* styling of the code
* and performance expectations

Passing the table `metadata` is crucial since it allows the model to generate consistent code that follows the structure of the metadata. 

Ensure that your problem statement is very clear, concise, and contains all relevant context.

In [10]:
metadata = """
          A table of bike trips, with the following columns:

          tripduration: Trip Duration (in seconds).
          starttime: Start Time, in NYC local time.
          stoptime: Stop Time, in NYC local time 
          start_station_id: Start Station ID
          start_station_name: Start Station Name
          start_station_latitude: Start Station Latitude
          start_station_longitude: Start Station Longitude
          end_station_id: End Station ID
          end_station_name: End Station Name
          end_station_latitude: End Station Latitude
          end_station_longitude: End Station Longitude
          bikeid: the Bike's unique ID number
          usertype: User Type (Customer = 24-hour pass or 7-day pass user, Subscriber = Annual Member)
          birth_year: user's year of birth
          gender: user's gender (unknown, male, female)
          customer_plan: The name of the plan that determines the rate charged for the trip

            """

language = "BigQuery"

problem = """generate the following query: \n
            - return all columns for 1000 records where tripduration is not null. \n
          """

PROMPT = f"""Write a {language} query to {problem}.
          use this as the table metadata: {metadata}.
          """

print(PROMPT)

Write a BigQuery query to generate the following query: 

            - return all columns for 1000 records where tripduration is not null. 

          .
          use this as the table metadata: 
          A table of bike trips, with the following columns:

          tripduration: Trip Duration (in seconds).
          starttime: Start Time, in NYC local time.
          stoptime: Stop Time, in NYC local time 
          start_station_id: Start Station ID
          start_station_name: Start Station Name
          start_station_latitude: Start Station Latitude
          start_station_longitude: Start Station Longitude
          end_station_id: End Station ID
          end_station_name: End Station Name
          end_station_latitude: End Station Latitude
          end_station_longitude: End Station Longitude
          bikeid: the Bike's unique ID number
          usertype: User Type (Customer = 24-hour pass or 7-day pass user, Subscriber = Annual Member)
          birth_year: user's year 

**Note:** replace `your_project.your_dataset.your_table` with `bigquery-public-data.new_york_citibike.citibike_trips`

In [11]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents=PROMPT,
    config=GenerateContentConfig(
        # tools=[code_execution_tool],
        temperature=0,
    ),
)

print(response.text)

```sql
SELECT
    *
  FROM
    `your_project.your_dataset.your_table`  -- Replace with your actual table information
  WHERE tripduration IS NOT NULL
LIMIT 1000
```

**Explanation:**

* **`SELECT *`**:  This selects all columns from the table.
* **`FROM your_project.your_dataset.your_table`**:  This specifies the table you want to query.  **Crucially, you MUST replace `your_project`, `your_dataset`, and `your_table` with the actual project ID, dataset name, and table name where your bike trip data is stored.**  You can find this information in the BigQuery console.
* **`WHERE tripduration IS NOT NULL`**: This filters the results to include only rows where the `tripduration` column has a value (i.e., it's not null).
* **`LIMIT 1000`**: This limits the result set to the first 1000 rows that satisfy the `WHERE` clause.  This is important for performance and cost control, especially when working with large datasets.

**How to use this query:**

1. **Open the BigQuery console:** Go to the G

In [16]:
problem = """generate the following query: \n
            - Show me the duration and subscriber type for the ten longest trips. \n
          """

PROMPT = f"""Write a {language} query to {problem}.
          use this as the table metadata: {metadata}.
          """

# print(PROMPT)

In [13]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents=PROMPT,
    config=GenerateContentConfig(
        # tools=[code_execution_tool],
        temperature=0,
    ),
)

print(response.text)

```sql
SELECT
    tripduration,
    usertype
  FROM
    `bigquery-public-data.new_york_citibike.citibike_trips`
ORDER BY
  tripduration DESC
LIMIT 10
```

**Explanation:**

*   **`SELECT tripduration, usertype`**: This specifies the columns you want to retrieve: the trip duration and the user type.
*   **`FROM \`bigquery-public-data.new_york_citibike.citibike_trips\``**:  This indicates the table you're querying from the BigQuery public dataset.  The backticks are important because the table name contains periods.
*   **`ORDER BY tripduration DESC`**: This sorts the results in descending order based on the `tripduration` column, so the longest trips appear first.
*   **`LIMIT 10`**: This limits the output to the top 10 rows, giving you the ten longest trips.

This query directly addresses the prompt's request to show the duration and subscriber type (which is represented by the `usertype` column) for the ten longest trips.



### generate multiple queries with single prompt

In [17]:
language = "BigQuery"

problem = """solve following queries: \n
            - Total number of trips taken. \n
            - Total number of seconds taken furing trips. \n
            - Average trip duration in seconds. \n
            - Most popular station trips start from. \n
            - Most recent trip completed. \n
          """

PROMPT = f"""Write a {language} query to {problem}.
          use this as the table metadata: {metadata}.
          Generate each query as a separate query separated with a comment.
              """

print(PROMPT)

Write a BigQuery query to solve following queries: 

            - Total number of trips taken. 

            - Total number of seconds taken furing trips. 

            - Average trip duration in seconds. 

            - Most popular station trips start from. 

            - Most recent trip completed. 

          .
          use this as the table metadata: 
          A table of bike trips, with the following columns:

          tripduration: Trip Duration (in seconds).
          starttime: Start Time, in NYC local time.
          stoptime: Stop Time, in NYC local time 
          start_station_id: Start Station ID
          start_station_name: Start Station Name
          start_station_latitude: Start Station Latitude
          start_station_longitude: Start Station Longitude
          end_station_id: End Station ID
          end_station_name: End Station Name
          end_station_latitude: End Station Latitude
          end_station_longitude: End Station Longitude
          bikeid: 

In [15]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents=PROMPT,
    config=GenerateContentConfig(
        # tools=[code_execution_tool],
        temperature=0,
    ),
)

print(response.text)

```sql
-- Total number of trips taken.
SELECT COUNT(*) AS total_trips
FROM `your_project.your_dataset.your_table`;

-- Total number of seconds taken during trips.
SELECT SUM(tripduration) AS total_seconds
FROM `your_project.your_dataset.your_table`;

-- Average trip duration in seconds.
SELECT AVG(tripduration) AS average_duration_seconds
FROM `your_project.your_dataset.your_table`;

-- Most popular station trips start from.
SELECT start_station_name, COUNT(*) AS trip_count
FROM `your_project.your_dataset.your_table`
GROUP BY start_station_name
ORDER BY trip_count DESC
LIMIT 1;

-- Most recent trip completed.
SELECT MAX(stoptime) AS most_recent_trip_end_time
FROM `your_project.your_dataset.your_table`;
```

**Explanation:**

*   **`your_project.your_dataset.your_table`**:  Replace this placeholder with the actual project ID, dataset name, and table name where your bike trip data is stored in BigQuery.

*   **Total number of trips:** `COUNT(*)` counts all rows in the table, representing

### add addiitonal requirements

In [18]:
additional_requirement = """
            - The query should be efficient and scalable, as the customer table may contain millions of rows. \n
            - The query should be easy to read and maintain.
            """

PROMPT = f"""Write a {language} query to {problem}.
          use this as the table metadata: {metadata}.
          Here are some additional requirement for the query: {additional_requirement}.
          Generate each query as a separate query separated with a comment.
              """

print(PROMPT)

Write a BigQuery query to solve following queries: 

            - Total number of trips taken. 

            - Total number of seconds taken furing trips. 

            - Average trip duration in seconds. 

            - Most popular station trips start from. 

            - Most recent trip completed. 

          .
          use this as the table metadata: 
          A table of bike trips, with the following columns:

          tripduration: Trip Duration (in seconds).
          starttime: Start Time, in NYC local time.
          stoptime: Stop Time, in NYC local time 
          start_station_id: Start Station ID
          start_station_name: Start Station Name
          start_station_latitude: Start Station Latitude
          start_station_longitude: Start Station Longitude
          end_station_id: End Station ID
          end_station_name: End Station Name
          end_station_latitude: End Station Latitude
          end_station_longitude: End Station Longitude
          bikeid: 

In [19]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents=PROMPT,
    config=GenerateContentConfig(
        # tools=[code_execution_tool],
        temperature=0,
    ),
)

print(response.text)

```sql
-- Total number of trips taken.
SELECT COUNT(*) AS total_trips
FROM `your_project.your_dataset.your_table`;

-- Total number of seconds taken during trips.
SELECT SUM(tripduration) AS total_seconds
FROM `your_project.your_dataset.your_table`;

-- Average trip duration in seconds.
SELECT AVG(tripduration) AS average_trip_duration
FROM `your_project.your_dataset.your_table`;

-- Most popular station trips start from.
SELECT start_station_name, COUNT(*) AS trip_count
FROM `your_project.your_dataset.your_table`
GROUP BY start_station_name
ORDER BY trip_count DESC
LIMIT 1;

-- Most recent trip completed.
SELECT MAX(stoptime) AS most_recent_trip_completion_time
FROM `your_project.your_dataset.your_table`;
```

**Explanation and Justification:**

*   **Efficiency and Scalability:**  All queries use aggregate functions (COUNT, SUM, AVG, MAX) which are highly optimized in BigQuery.  The `GROUP BY` and `ORDER BY` operations in the "Most popular station" query are also handled efficiently 

## SQL Metadata & Performance

> example prompting strategy using fictious SQL table

In [20]:
problem_statement = """
                    You are a developer working on an e-commerce platform.
                    The marketing team has requested a report on the total number of orders and the average order \
                    value for each product category for the past month.
                    Your task is to generate a SQL queries to retrieve the total number of orders and the average order \
                    value for each product category for the orders placed in the:
                    1) past month,
                    2) given data range,
                    3) end of each month for given year,
                    4) christmas and new year's eve.
                    """

table_metadata = """
                 - **Orders:**
                    - `OrderID` (integer)
                    - `ProductID` (integer)
                    - `ProductName` (string)
                    - `Category` (string)
                    - `OrderDate` (date)
                    - `OrderAmount` (decimal)
                """

language = "SQL"

code_style = f"""
            Write a {language} query that follows best practices, is readable, and well-commented.
             """

performance_requirement = """
                          Optimize the query for performance considering the potential size of the "Orders" table.
                          Consider using appropriate indexing if necessary.
                          """

PROMPT = f""" Solve the following: {problem_statement}. The given table metadata is : {table_metadata} .
          Follow the following code style: {code_style} . The following performance requirement is: {performance_requirement} .
          """

print(PROMPT)

 Solve the following: 
                    You are a developer working on an e-commerce platform.
                    The marketing team has requested a report on the total number of orders and the average order                     value for each product category for the past month.
                    Your task is to generate a SQL queries to retrieve the total number of orders and the average order                     value for each product category for the orders placed in the:
                    1) past month,
                    2) given data range,
                    3) end of each month for given year,
                    4) christmas and new year's eve.
                    . The given table metadata is : 
                 - **Orders:**
                    - `OrderID` (integer)
                    - `ProductID` (integer)
                    - `ProductName` (string)
                    - `Category` (string)
                    - `OrderDate` (date)
                    - `OrderAm

In [7]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents=PROMPT,
    config=GenerateContentConfig(
        # tools=[code_execution_tool],
        temperature=0,
    ),
)

print(response.text)

```sql
 -- 1. Total orders and average order value for each product category for the past month.
 

 SELECT
  Category,
  COUNT(DISTINCT OrderID) AS TotalOrders,
  AVG(OrderAmount) AS AverageOrderValue
 FROM
  Orders
 WHERE
  OrderDate >= DATE_SUB(CURDATE(), INTERVAL 1 MONTH)
 GROUP BY
  Category
 ORDER BY
  Category;
 

 -- Optimization notes:
 --  - Ensure that there is an index on the `OrderDate` column to speed up the filtering process.
 --  - An index on the `Category` column would also improve the performance of the GROUP BY clause.
 

 -- 2. Total orders and average order value for each product category for a given date range.
 

 SELECT
  Category,
  COUNT(DISTINCT OrderID) AS TotalOrders,
  AVG(OrderAmount) AS AverageOrderValue
 FROM
  Orders
 WHERE
  OrderDate BETWEEN '2023-01-01' AND '2023-01-31' --  Replace with your desired start and end dates
 GROUP BY
  Category
 ORDER BY
  Category;
 

 -- Optimization notes:
 --  - Similar to the previous query, ensure that there is an